In [13]:
import torch
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
# import scipy
# import umap
import random
# import itertools
import tsl
import numpy as np
# import pandas as pd
import networkx as nx
import torch_geometric
import lightning as L
import matplotlib.pyplot as plt

from tsl.datasets import PvUS
from tsl.data.datamodule import SpatioTemporalDataModule, TemporalSplitter
from tsl.data.preprocessing import StandardScaler

# from dataset.NCI1_dataset import NCI1
# from tqdm import trange
# from sklearn.model_selection import train_test_split

from models.DynGraphESN import DynGESNModel

from DMD.dmd import KANN

seed = 42
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

for more datasets:

https://github.com/dtortorella/dyngraphesn/tree/master

In [2]:
dataset = PvUS(root="/dataset", zones=['west'])

In [3]:
print(dataset)

PvUS-west(length=52560, n_nodes=1082, n_channels=1)


In [3]:
print(f"Default similarity: {dataset.similarity_score}")
print(f"Available similarity options: {dataset.similarity_options}")
print("==========================================")

sim = dataset.get_similarity("distance")  # or dataset.compute_similarity()

# print("Similarity matrix W:")
# pd.DataFrame(sim)

Default similarity: distance
Available similarity options: {'correntropy', 'distance'}


In [3]:
connectivity = dataset.get_connectivity(threshold=0.1,
                                        include_self=False,
                                        normalize_axis=1,
                                        layout="edge_index")

In [14]:
torch_dataset = tsl.data.SpatioTemporalDataset(target=dataset.dataframe(),
                                      connectivity=connectivity,
                                      mask=dataset.mask,
                                      horizon=1,
                                      window=12,
                                      stride=1)
print(torch_dataset)

SpatioTemporalDataset(n_samples=52548, n_nodes=1082, n_channels=1)


In [5]:
sample = torch_dataset[0].to(device)
print(sample)

Data(
  input=(x=[t=12, n=1082, f=1], edge_index=[2, e=124778], edge_weight=[e=124778]),
  target=(y=[t=12, n=1082, f=1]),
  has_mask=False
)


In [10]:
print(sample.pattern)

{'x': 't n f', 'edge_index': '2 e', 'edge_weight': 'e', 'y': 't n f'}


In [15]:
scalers = {'target': StandardScaler(axis=(0, 1))}

# Split data sequentially:
#   |------------ dataset -----------|
#   |--- train ---|- val -|-- test --|
splitter = TemporalSplitter(val_len=0.1, test_len=0.2)

dm = SpatioTemporalDataModule(
    dataset=torch_dataset,
    scalers=scalers,
    splitter=splitter,
    batch_size=64,
)

dm.setup()
print(dm)

SpatioTemporalDataModule(train_len=37824, val_len=4191, test_len=10509, scalers=[target], batch_size=64)


In [10]:
input_size = sample.input.x.shape[2]

model = DynGESNModel(input_size=input_size,
                reservoir_size=100,
                input_scaling=1.,
                reservoir_layers=1,
                leaking_rate=0.9,
                spectral_radius=0.9,
                density=0.5,
                reservoir_activation='tanh',
                alpha_decay=False).to(device)

In [13]:
sample.input

StorageView(x=[12, 1082, 1], edge_index=[2, 124778], edge_weight=[124778])

In [12]:
model(sample.input.x, sample.input.edge_index, sample.input.edge_weight).shape

torch.Size([12, 1082, 100])

In [ ]:
class LinearRegression(pl.LightningModule):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        return self.linear(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.functional.mse_loss(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
